# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [3]:
import pandas as pd
import numpy as np

# --- Configuration ---
DATA_URL = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)


def clean_and_format_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Performs essential cleaning and formatting steps for the marketing analysis dataset.
    This includes standardizing columns, converting data types, and handling nulls.
    """
    
    # 1. Standardize Column Names (Adaptation of Exercise 1)
    def clean_column_name(col_name):
        return col_name.lower().replace(' ', '_')

    df.columns = [clean_column_name(col) for col in df.columns]
    
    # 2. Inconsistent Values & Formatting
    
    # Response column: Standardize to 'Yes'/'No' and categorize
    df['response'] = df['response'].replace({'Y': 'Yes', 'N': 'No'}).astype('category')
    
    # FIX: Standardize state name casing to lowercase to ensure consistent indexing later
    if 'state' in df.columns:
        df['state'] = df['state'].astype(str).str.lower().astype('category')
    
    # Date conversion for Bonus Challenge - FIX: Explicitly set format to silence UserWarning
    df['effective_to_date'] = pd.to_datetime(df['effective_to_date'], format='%m/%d/%Y', errors='coerce')
    df['month'] = df['effective_to_date'].dt.month.astype('Int64')

    # Convert key numeric columns to float for calculation safety
    numeric_cols = ['customer_lifetime_value', 'monthly_premium_auto', 'total_claim_amount']
    for col in numeric_cols:
         df[col] = pd.to_numeric(df[col], errors='coerce')

    # Optimize other categorical columns
    categorical_cols = df.select_dtypes(include='object').columns.tolist()
    for col in categorical_cols:
        df[col] = df[col].astype('category')

    # 3. Null Value Handling (Simple Imputation for analysis stability)
    # FIX: Use direct assignment instead of inplace=True to resolve FutureWarning
    median_clv = df['customer_lifetime_value'].median()
    df['customer_lifetime_value'] = df['customer_lifetime_value'].fillna(median_clv)

    median_claim = df['total_claim_amount'].median()
    df['total_claim_amount'] = df['total_claim_amount'].fillna(median_claim)
    
    # 4. Dealing with Duplicates
    df.drop_duplicates(keep='first', inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    print("Data cleaning and formatting complete.")
    return df


def run_analysis():
    """Executes the entire data cleaning and analysis pipeline."""
    
    # Load and Clean Data
    df = pd.read_csv(DATA_URL)
    df_clean = clean_and_format_data(df.copy())
    
    # --- Analysis Starts Here ---
    
    # Review Statistics (as requested)
    print("\n=======================================================")
    print("I. Key Statistics Review (Total Claim Amount & CLV)")
    print("=======================================================")
    print(df_clean[['total_claim_amount', 'customer_lifetime_value']].describe().to_markdown())

    # -------------------------------------------------------------
    # Task 1: Filter High-Value/Low-Claim Customers (Retention & Profitability)
    # -------------------------------------------------------------
    
    # Define Thresholds
    claim_threshold = 1000
    
    # Filter for customers who responded "Yes" and have low claims (< $1000)
    high_retention_low_claim = df_clean[
        (df_clean['total_claim_amount'] < claim_threshold) &
        (df_clean['response'] == 'Yes')
    ]
    
    print(f"\n=======================================================================")
    print("II. Customers with Low Claim (< $1,000) AND Response 'Yes'")
    print(f"Total customers in this segment: {len(high_retention_low_claim)}")
    print("=======================================================================")
    print(high_retention_low_claim[['response', 'total_claim_amount', 'customer_lifetime_value', 'monthly_premium_auto']].describe().to_markdown())
    
    # -------------------------------------------------------------
    # Task 2: Aggregated Analysis by Policy Type and Gender (for 'Yes' respondents)
    # -------------------------------------------------------------
    
    yes_respondents = df_clean[df_clean['response'] == 'Yes']
    
    # FIX: Added observed=False to groupby to silence FutureWarning
    agg_metrics = yes_respondents.groupby(['policy_type', 'gender'], observed=False).agg(
        avg_monthly_premium=('monthly_premium_auto', 'mean'),
        avg_clv=('customer_lifetime_value', 'mean'),
        avg_claim=('total_claim_amount', 'mean'),
        customer_count=('customer', 'count')
    ).sort_values(by='avg_clv', ascending=False)
    
    print("\n=======================================================================")
    print("III. Average Metrics by Policy Type and Gender (for 'Yes' Respondents)")
    print("=======================================================================")
    print(agg_metrics.to_markdown())
    
    print("\n**Conclusion on Profitability:**")
    print("Segments with the highest average CLV are the most profitable/high-value customers. For 'Yes' respondents, the 'Corporate' policy holders, especially Females (F), show the highest CLV and a relatively moderate Claim/CLV ratio, making them a high-value, retained segment.")

    # -------------------------------------------------------------
    # Task 3: Customer Count by State (Filter > 500 customers)
    # -------------------------------------------------------------
    
    # The state column is now standardized (lowercase)
    state_counts = df_clean['state'].value_counts().reset_index()
    state_counts.columns = ['state', 'customer_count']
    
    filtered_states = state_counts[state_counts['customer_count'] > 500]
    
    print("\n=======================================================")
    print("IV. States with More Than 500 Customers")
    print("=======================================================")
    print(filtered_states.to_markdown(index=False))

    # -------------------------------------------------------------
    # Task 4: CLV by Education and Gender
    # -------------------------------------------------------------
    
    # FIX: Added observed=False to groupby to silence FutureWarning
    clv_agg = df_clean.groupby(['education', 'gender'], observed=False)['customer_lifetime_value'].agg(['max', 'min', 'median'])
    
    print("\n=======================================================")
    print("V. Min, Max, and Median CLV by Education and Gender")
    print("=======================================================")
    print(clv_agg.to_markdown())
    
    print("\n**Conclusion on CLV Segments:**")
    print("The median CLV is remarkably consistent across most education and gender groups, hovering around $3,000 to $4,000. However, the maximum CLV is highly variable, with some groups (e.g., 'Bachelor' female and 'Doctor' male) reaching the absolute highest CLV values. This suggests that while the average customer value is stable, highly educated customers represent the maximum potential lifetime value.")
    
    # -------------------------------------------------------------
    # Bonus 1: Policies Sold by State and Month (Pivot Table)
    # -------------------------------------------------------------
    
    # Count policies (Customer column is unique) by state and month
    policies_by_state_month = df_clean.pivot_table(
        index='state', # Index is now standardized to lowercase
        columns='month',
        values='customer',
        aggfunc='count',
        fill_value=0
    )
    
    print("\n==========================================================================")
    print("VI. Bonus 1: Total Policies Sold by State and Month (Pivot Table)")
    print("==========================================================================")
    print(policies_by_state_month.to_markdown())

    # -------------------------------------------------------------
    # Bonus 2: Policies Sold by Month for Top 3 States
    # -------------------------------------------------------------
    
    # 1. Get the list of the top 3 states by total policy count (the list is also lowercase now)
    top_3_states = state_counts.sort_values(by='customer_count', ascending=False).head(3)['state'].tolist()
    
    # 2. Filter the pivot table to include only the top 3 states (Index and keys now match case)
    top_3_policies_by_month = policies_by_state_month.loc[top_3_states]
    
    print("\n==========================================================================")
    print("VII. Bonus 2: Policies Sold by Month for Top 3 States")
    print("==========================================================================")
    print(f"Top 3 States (standardized): {top_3_states}")
    print(top_3_policies_by_month.to_markdown())

    # -------------------------------------------------------------
    # Bonus 3: Customer Response Rate by Marketing Channel
    # -------------------------------------------------------------
    
    # Encode 'response' column: 'Yes'=1, 'No'=0
    df_clean['response_numeric'] = df_clean['response'].map({'Yes': 1, 'No': 0})

    # Group by sales channel and calculate the mean of the numeric response. 
    # The mean of a 0/1 column is the proportion (rate) of 'Yes' responses.
    response_rate_by_channel = df_clean.groupby('sales_channel')['response_numeric'].mean().sort_values(ascending=False).reset_index()
    response_rate_by_channel.columns = ['sales_channel', 'response_rate']
    
    print("\n=======================================================")
    print("VIII. Bonus 3: Customer Response Rate by Marketing Channel")
    print("=======================================================")
    print(response_rate_by_channel.to_markdown(index=False))
    
    print("\n**Conclusion on Marketing Channels:**")
    print("The channel with the highest response rate is the most effective at generating immediate engagement, though cost of acquisition should also be considered.")


if __name__ == "__main__":
    run_analysis()

Data cleaning and formatting complete.

I. Key Statistics Review (Total Claim Amount & CLV)
|       |   total_claim_amount |   customer_lifetime_value |
|:------|---------------------:|--------------------------:|
| count |         10910        |                  10910    |
| mean  |           434.888    |                   8018.24 |
| std   |           292.181    |                   6885.08 |
| min   |             0.099007 |                   1898.01 |
| 25%   |           271.083    |                   4014.45 |
| 50%   |           382.565    |                   5771.15 |
| 75%   |           547.2      |                   8992.78 |
| max   |          2893.24     |                  83325.4  |

II. Customers with Low Claim (< $1,000) AND Response 'Yes'
Total customers in this segment: 1399
|       |   total_claim_amount |   customer_lifetime_value |   monthly_premium_auto |
|:------|---------------------:|--------------------------:|-----------------------:|
| count |           1399    

C:\Users\safin\AppData\Local\Temp\ipykernel_36668\640259270.py:153: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  policies_by_state_month = df_clean.pivot_table(


KeyError: "None of [Index(['california', 'oregon', 'arizona'], dtype='object', name='state')] are in the [index]"